In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium.features import DivIcon
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import make_pipeline as imb_make_pipeline 
from imblearn.over_sampling import SMOTE

# Match Gas Stations with EV

In [ ]:
# load gas station data
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Capstone-KPMG/Preprocessing/Data
!ls
df_use = pd.read_csv("main_dataset_all_interstate_all_route_10mi_w_income.csv")
df_use.head()

Mounted at /content/drive/
/content/drive/My Drive/Capstone-KPMG/Preprocessing/Data
gas_station_w_dist_to_exit_wholeI5I90I82.csv
main_dataset_all_interstate_all_route_10mi.csv
main_dataset_all_interstate_all_route_10mi_w_income.csv
main_dataset_all_interstate.csv
main_dataset.csv
wa_EV_stations.csv
wa_washington_zip_codes_geo.json


,Unnamed: 0,gas_key,gas_name,gas_lat,gas_long,attr_cnt_1mile,attr_cnt_5mile,attr_name,attr_lat,attr_long,...,nri_zipcode,census_tract_area,census_tract_category,traff_cnt_10m_avg,traff_cnt_10m_max,traff_cnt_5m_avg,traff_cnt_5m_max,zip,zip_income,household_median_income
0,0,0,24 Hour Express Fuel,48.393760,-122.329806,0,1,Skagit County Fairgrounds,48.408145,-122.340118,...,98273.0,0.002191,2.0,21209.859150,72000,23733.802820,72000,98273.0,98273.0,64699
1,1,1,76,47.191712,-120.916225,0,1,KBHR Set,47.223241,-120.993224,...,NaN,0.000584,2.0,11678.727270,36000,9901.025641,34000,98922.0,98922.0,61071
2,2,2,76,47.238704,-122.357577,0,4,Wagon Trail,47.275437,-122.308897,...,98424.0,0.001614,2.0,39766.900000,219000,43117.857140,219000,98424.0,98424.0,78243
3,3,3,76,48.435227,-122.291324,0,1,Skagit County Fairgrounds,48.408145,-122.340118,...,NaN,0.001533,2.0,20470.068490,72000,23101.204820,72000,98273.0,98273.0,64699
4,4,4,76,47.839608,-120.014523,0,0,NaN,NaN,NaN,...,NaN,0.018331,0.0,5653.947368,12000,6723.076923,12000,98817.0,98817.0,71279


In [ ]:
df_use = df_use.drop('Unnamed: 0', axis = 1)

In [ ]:
df_use.columns

Index(['gas_key', 'gas_name', 'gas_lat', 'gas_long', 'attr_cnt_1mile',
       'attr_cnt_5mile', 'attr_name', 'attr_lat', 'attr_long',
       'distance_to_nearest_attr', 'crime_coord', 'crime_county',
       'crime_population', 'violent_crime', 'murder_nonnegligent_manslaughter',
       'Rape1', 'Robbery', 'aggravated_assault', 'property_crime', 'Burglary',
       'larceny_theft', 'motor_vehicle_theft', 'Arson', 'total_crime',
       'exit_name', 'exit_lat', 'exit_long', 'distance_to_nearest_exit',
       'highway', 'num_EV_in_2_miles_of_gas', 'num_EV_in_5_miles_of_gas',
       'num_EV_in_10_miles_of_gas', 'num_EV_in_20_miles_of_gas',
       'num_EV_in_50_miles_of_gas', 'Closest_EV_Station_name',
       'Closest_EV_Station_lat', 'Closest_EV_Station_long',
       'distance_to_closest_ev_station', 'nri_geoid', 'nri_county',
       'nri_population', 'nri_build_value', 'nri_agri_value', 'nri_area',
       'nri_risk_score', 'nri_risk_rating', 'nri_intpt_lat', 'nri_intpt_long',
       'nri_zi

# Preprocessing

In [ ]:
df_use['census_tract_category'].value_counts()

2.0    1562
1.0     332
0.0     314
Name: census_tract_category, dtype: int64

In [ ]:
df_use["household_median_income"] == "-"

0       False
1       False
2       False
3       False
4       False
        ...  
2206    False
2207    False
2208    False
2209    False
2210    False
Name: household_median_income, Length: 2211, dtype: bool

In [ ]:
df_use["household_median_income"] = df_use["household_median_income"].apply(lambda x: np.nan if x == '-' else x)

In [ ]:
pd.to_numeric(df_use["household_median_income"]).unique()

array([ 64699.,  61071.,  78243.,  71279.,  88625.,  57922., 134424.,
        58018., 136208., 102818.,  84489.,  79127.,     nan, 105115.,
        40313.,  61213.,  75099.,  67142.,  53093.,  97374.,  59097.,
        96165., 122388., 143106.,  95682.,  71093.,  48522.,  64896.,
        58611.,  90636.,  95190., 104406.,  50037.,  97106.,  93405.,
        72711.,  96121.,  82241.,  92216.,  80586.,  66237.,  86594.,
       109031.,  60388., 156382.,  85541.,  73702.,  70330.,  84957.,
        72397.,  79943.,  87320.,  66021., 188500., 101847.,  81471.,
        71546.,  59415.,  74179.,  77036.,  88446.,  78845.,  48041.,
        62440.,  87882.,  82684.,  45972.,  70660.,  85875.,  71059.,
        56754., 100955., 107109.,  69458.,  67170., 158168.,  80633.,
        66685., 103421.,  58961.,  71014.,  77108., 102135.,  70374.,
        81776., 135225., 127724.,  90844.,  59000.,  59208.,  60255.,
        88893.,  59357.,  42591.,  59058.,  85943.,  61863.,  71121.,
       106059.,  437

# Fill N/A
1. nri_risk_score: mean
2. household_median_income: min
3. census_tract_category: NA as 'city'
4. crime_population: min


In [ ]:
# drop not useful columns
df_use_cleaned = df_use.drop(['gas_name',
                              'attr_name', 'attr_lat', 'attr_long','distance_to_nearest_attr', 'attr_cnt_5mile',
                              'crime_coord', 'crime_county',
                              'exit_name', 'exit_lat', 'exit_long', 'distance_to_nearest_exit', # used this in optimization
                              'Closest_EV_Station_name', 'Closest_EV_Station_lat', 'Closest_EV_Station_long', 'num_EV_in_2_miles_of_gas',
                              'nri_geoid', 'nri_county', 'nri_zipcode', 'nri_population', 'nri_intpt_lat', 'nri_intpt_long','nri_build_value', 'nri_agri_value', 'nri_area', 'nri_risk_rating',
                              'census_tract_area', 'distance_to_closest_ev_station',
                              'num_EV_in_5_miles_of_gas', 'num_EV_in_10_miles_of_gas', 'num_EV_in_20_miles_of_gas', 'num_EV_in_50_miles_of_gas',
                              'traff_cnt_10m_avg', 'traff_cnt_10m_max', 'traff_cnt_5m_avg', 'traff_cnt_5m_max',
                              'zip', 'zip_income'], axis = 1)


# for na crime we take min
crime_list = ['violent_crime', 'murder_nonnegligent_manslaughter','Robbery', 'Rape1','aggravated_assault', 'Burglary', 'Arson', 'property_crime', 'larceny_theft', 'motor_vehicle_theft', 'total_crime']
df_use_cleaned[crime_list] = df_use_cleaned[crime_list].apply(lambda x: x/df_use_cleaned['crime_population'], axis = 0)
for i in crime_list:
  df_use_cleaned[i]= df_use_cleaned[i].fillna(df_use_cleaned[i].min())

# for na nri we take average
# nri_list = ['nri_build_value', 'nri_agri_value', 'nri_area', 'nri_risk_score']
# for i in nri_list:
df_use_cleaned['nri_risk_score']= df_use_cleaned['nri_risk_score'].fillna(df_use_cleaned['nri_risk_score'].mean())

# for na median income we take min (some areas has no median income record may infer hard-to-reach areas)
df_use_cleaned["household_median_income"] = df_use_cleaned["household_median_income"].apply(lambda x: np.nan if x == '-' else x)
df_use_cleaned["household_median_income"] = pd.to_numeric(df_use_cleaned["household_median_income"])
df_use_cleaned['household_median_income'] = df_use_cleaned['household_median_income'].fillna(df_use_cleaned['household_median_income'].min())

# # nri
#df_use_cleaned['nri_risk_rating'] = df_use_cleaned['nri_risk_rating'].fillna("Insufficient Data")

# def nri_type(x):
#   if x =='Very Low':
#     return -2
#   elif x == 'Relatively Low':
#     return -1
#   elif x == 'Relatively Moderate' or x == 'Insufficient Data' :
#     return 0
#   elif x == 'Relatively High':
#     return 1 
#   elif x == 'Very High':
#     return 2

# df_use_cleaned['nri_risk_rating'] = df_use_cleaned['nri_risk_rating'].apply(lambda x: nri_type(x))


# census tract
df_use_cleaned['census_tract_category'] = df_use_cleaned['census_tract_category'].fillna(2) # 2 is city
df_use_cleaned['census_tract_city'] = df_use_cleaned['census_tract_category'].apply(lambda x: 0 if x<2 else 1)

# drop additional unuseful columns
df_use_cleaned = df_use_cleaned.drop(['crime_population'], axis = 1)


df_use_cleaned.isna().sum()

gas_key                               0
gas_lat                               0
gas_long                              0
attr_cnt_1mile                        0
violent_crime                         0
murder_nonnegligent_manslaughter      0
Rape1                                 0
Robbery                               0
aggravated_assault                    0
property_crime                        0
Burglary                              0
larceny_theft                         0
motor_vehicle_theft                   0
Arson                                 0
total_crime                           0
highway                             555
nri_risk_score                        0
census_tract_category                 0
household_median_income               0
census_tract_city                     0
dtype: int64

In [ ]:
# %cd /content/drive/My Drive/Capstone-KPMG/Preprocessing/Data

# df_use_cleaned.to_csv('cleaned_main_data.csv')

In [ ]:
df_use_cleaned.dtypes

gas_key                               int64
gas_lat                             float64
gas_long                            float64
attr_cnt_1mile                        int64
violent_crime                       float64
murder_nonnegligent_manslaughter    float64
Rape1                               float64
Robbery                             float64
aggravated_assault                  float64
property_crime                      float64
Burglary                            float64
larceny_theft                       float64
motor_vehicle_theft                 float64
Arson                               float64
total_crime                         float64
highway                              object
nri_risk_score                      float64
census_tract_category               float64
household_median_income             float64
census_tract_city                     int64
dtype: object

In [ ]:
df_use_cleaned.head()

,gas_key,gas_lat,gas_long,attr_cnt_1mile,violent_crime,murder_nonnegligent_manslaughter,Rape1,Robbery,aggravated_assault,property_crime,Burglary,larceny_theft,motor_vehicle_theft,Arson,total_crime,highway,nri_risk_score,census_tract_category,household_median_income,census_tract_city
0,0,48.393760,-122.329806,0,0.001397,0.000027,0.000319,0.000373,0.000679,0.031173,0.003699,0.025173,0.002302,0.000093,0.065234,I5,25.815695,2.0,64699.0,1
1,1,47.191712,-120.916225,0,0.002057,0.000000,0.000494,0.000247,0.001316,0.029984,0.003949,0.024760,0.001275,0.000123,0.064204,I90,26.911803,2.0,61071.0,1
2,2,47.238704,-122.357577,0,0.006015,0.000040,0.000604,0.001387,0.003983,0.041052,0.006588,0.028933,0.005532,0.000316,0.094450,I5,35.360714,2.0,78243.0,1
3,3,48.435227,-122.291324,0,0.001397,0.000027,0.000319,0.000373,0.000679,0.031173,0.003699,0.025173,0.002302,0.000093,0.065234,I5,26.911803,2.0,64699.0,1
4,4,47.839608,-120.014523,0,0.001796,0.000029,0.000464,0.000290,0.001014,0.018486,0.001912,0.015414,0.001159,0.000029,0.040593,NaN,26.911803,0.0,71279.0,0


In [ ]:
%cd /content/drive/My Drive/Capstone-KPMG/Modeling
result_list = pd.read_csv('optimization_result.csv')['gas_key'].to_list()
df_test = df_use_cleaned.iloc[df_use[df_use['gas_key'].isin(result_list)].index.tolist()].drop(['highway','census_tract_category'], axis = 1)


print("test:", df_test.shape)
print("total:", df_use_cleaned.shape)

/content/drive/My Drive/Capstone-KPMG/Modeling
test: (25, 18)
total: (2211, 20)


In [ ]:
df_use_idx = df_use_cleaned.set_index(['gas_key', 'gas_lat', 'gas_long']).drop(['highway','census_tract_category'], axis = 1)

In [ ]:
from sklearn import preprocessing

x = df_use_idx.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_use_final = pd.DataFrame(x_scaled, index = df_use_idx.index,columns = df_use_idx.columns).reset_index()
df_use_final.head()

,gas_key,gas_lat,gas_long,attr_cnt_1mile,violent_crime,murder_nonnegligent_manslaughter,Rape1,Robbery,aggravated_assault,property_crime,Burglary,larceny_theft,motor_vehicle_theft,Arson,total_crime,nri_risk_score,household_median_income,census_tract_city
0,0,48.393760,-122.329806,0.0,0.181333,0.361641,0.166557,0.268567,0.084353,0.507692,0.440711,0.497893,0.397233,0.151809,0.474215,0.206491,0.265060,1.0
1,1,47.191712,-120.916225,0.0,0.298242,0.000000,0.257443,0.177907,0.261020,0.483994,0.470466,0.488176,0.205622,0.201127,0.464682,0.221595,0.244775,1.0
2,2,47.238704,-122.357577,0.0,1.000000,0.547683,0.315300,1.000000,1.000000,0.704488,0.784944,0.586442,1.000000,0.515128,0.744691,0.338019,0.340786,1.0
3,3,48.435227,-122.291324,0.0,0.181333,0.361641,0.166557,0.268567,0.084353,0.507692,0.440711,0.497893,0.397233,0.151809,0.474215,0.221595,0.265060,1.0
4,4,47.839608,-120.014523,0.0,0.252136,0.393778,0.241810,0.208881,0.177328,0.254932,0.227854,0.268056,0.183963,0.047229,0.246098,0.221595,0.301850,0.0


In [ ]:
def calculate_score(df, a = 1, b = 1, c = 1, d = 1):
  df[['attr_cnt_1mile', 'total_crime', 'nri_risk_score', 'household_median_income']]
  df['score'] = a*df['attr_cnt_1mile'] - b*df['total_crime'] - c*df['nri_risk_score'] - d*df['household_median_income']
  df['score'] = np.round((df['score']-df['score'].min())/(df['score'].max()-df['score'].min()),2)*100
  df['score'] = df['score'].astype(int)

  df_test = df.iloc[df_use[df_use['gas_key'].isin(result_list)].index.tolist()]

  return df_test

In [ ]:
df_final_score = calculate_score(df_use_final, a = 0.5, b=1, c=1, d=2)

In [ ]:
df_final_score.head()

,gas_key,gas_lat,gas_long,attr_cnt_1mile,violent_crime,murder_nonnegligent_manslaughter,Rape1,Robbery,aggravated_assault,property_crime,Burglary,larceny_theft,motor_vehicle_theft,Arson,total_crime,nri_risk_score,household_median_income,census_tract_city,score
45,45,47.093839,-122.625748,0.000000,1.000000,0.547683,0.315300,1.000000,1.000000,0.704488,0.784944,0.586442,1.000000,0.515128,0.744691,0.221595,0.425559,1.0,36
55,55,47.079480,-123.020796,0.000000,0.521303,0.192932,0.259165,0.562879,0.456853,0.502627,0.476152,0.468897,0.524034,0.196688,0.505272,0.221595,0.000000,0.0,75
119,119,48.940695,-122.485933,0.000000,0.352622,0.481167,0.152354,0.401998,0.305902,0.415656,0.375447,0.431388,0.164238,0.115420,0.406360,0.216141,0.308100,0.0,56
203,203,48.390487,-122.501414,0.076923,0.181333,0.361641,0.166557,0.268567,0.084353,0.507692,0.440711,0.497893,0.397233,0.151809,0.474215,0.221595,0.296253,0.0,56
403,403,48.365457,-122.429235,0.000000,0.181333,0.361641,0.166557,0.268567,0.084353,0.507692,0.440711,0.497893,0.397233,0.151809,0.474215,0.642996,0.265060,0.0,42


In [ ]:
#vdf_use[df_use['gas_key'].isin(result_list)].index.tolist()

In [ ]:
# df_final_score.iloc[df_use[df_use['gas_key'].isin(result_list)].index.tolist()]

In [ ]:
# df_test = df_final_score.iloc[df_use[df_use['gas_key'].isin(result_list)].index.tolist()]

# df_test['score']

In [ ]:
def plot(df): 

  center = 46.2735210909813, -122.89553326963093


  m = folium.Map(location=center, 
                zoom_start=10,
                width=400,height=950)

  # Same as before... go through each home in set, make circle, and add to map.
  # This time we add a color using price and the colormap object
  for i in range(len(df)):
      folium.Circle(
          location=[df.iloc[i]['gas_lat'], df.iloc[i]['gas_long']],
          radius=1000,
          fill=True,
          color = 'orange',
          fill_opacity=0.2
      ).add_to(m)

  # for i in range(len(df_ev)):
  #     folium.Circle(
  #         location=[df_ev.iloc[i]['Latitude'], df_ev.iloc[i]['Longitude']],
  #         radius=1000,
  #         fill=True,
  #         color = 'blue',
  #         fill_opacity=0.2
  #     ).add_to(m)

  for i in range(len(df)):
        folium.map.Marker([df.iloc[i]['gas_lat'], df.iloc[i]['gas_long']],
                        icon=DivIcon(
                            icon_size=(-10,15),
                            icon_anchor=(-10,14),
                            html=f'<div style="font-size: 14pt">%s</div>' % df.iloc[i]['score'],
                        )
                      ).add_to(m)

  return m

In [ ]:
plot(calculate_score(df_use_final, a = 1, b=1, c=1, d=1))

In [ ]:
plot(calculate_score(df_use_final, a = 0.5, b=1, c=1, d=1))

In [ ]:
plot(calculate_score(df_use_final, a = 0.5, b=1, c=1, d=2))

In [ ]:
plot(calculate_score(df_use_final, a =0.5, b=2, c=1, d=2))